In [3]:
import ollama
import base64

# Read and encode the image
image_path = "temp.png"  # Replace with your image file path
with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

# Construct the message with the image
messages = [
    {
        'role': 'user',
        'content': 'Please analyze the following image:',
        'images': ['temp.png']
    }
]

# Send the request to the 'llava' model
response = ollama.chat(model='llama3.2-vision', messages=messages)

# Output the response
print(response['message']['content'])


The image presents a comprehensive overview of the insurance policy's coverage for bushfires, highlighting both the areas covered and those excluded. The information is organized into three main sections: "We cover," "Extra Costs cover in the event of bushfire," and "We don't cover." 

*   **We Cover**
    *   Loss or damage caused by:
        *   Fire (includes bushfires)
        *   Heat, ash, soot, and smoke that are the direct result of a fire within 100 meters of the insured address
*   **Extra Costs Cover in the Event of Bushfire**
    *   The cost of replacing water in any tank, container, pool, spa, or other storage vessel where the water has been used to limit the spread of bushfires at the insured address or become contaminated due to fire retardant use at the insured address
    *   The cost of cleaning fire retardant off the building
*   **We Don't Cover**
    *   Loss or damage arising from:
        *   Arcing, scorching, melting, or cigarette burns unless a fire spreads f

In [9]:
result["choices"][0]["message"]["content"]

"The image is not provided, but based on the text, it appears to be a snippet from an insurance policy or a document related to property insurance, specifically discussing coverage for fire damage. The text outlines the terms and conditions of the policy, including what is covered and what is not, as well as the process for making a claim.\n\nHere are some key points that can be inferred from the text:\n\n* The policy covers damage caused by fire, but only for specific circumstances, such as a lightning strike or an electrical fault, and not for other types of fires, such as those caused by arson or electrical appliances.\n* The policy requires the insured to take steps to mitigate damage and prevent future occurrences, such as ensuring that electrical equipment is maintained and used properly.\n* The policy may also cover other types of damage, such as water damage or storm damage, but the specific details are not provided in the snippet.\n* The policy may have certain exclusions, suc

In [4]:
import fitz  # PyMuPDF
import tempfile
import os

class TemporaryImageFile:
    def __init__(self, pixmap):
        # Create a temporary file with .jpg suffix
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".jpg")
        self.filename = temp_file.name
        temp_file.close()  # Close the file so pixmap can write to it

        # Save pixmap to the temporary file in JPEG format
        pixmap.save(self.filename, output="jpg")

    def __del__(self):
        # Delete the temporary file when the object is garbage collected
        os.unlink(self.filename)

class PDFPageIterator:
    def __init__(self, pdf_data):
        """
        Accepts a PDF file path or bytes data and creates an iterator over JPEG images and text of each page.

        :param pdf_data: Path to the PDF file or PDF data in bytes.
        """
        # Open the PDF document
        if isinstance(pdf_data, str):
            self.doc = fitz.open(pdf_data)
        elif isinstance(pdf_data, bytes):
            self.doc = fitz.open("pdf", pdf_data)
        else:
            raise ValueError("pdf_data must be a file path or bytes data.")

        self.page_count = len(self.doc)
        self.current_page = 0  # Initialize the page counter

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_page >= self.page_count:
            raise StopIteration

        page = self.doc[self.current_page]
        self.current_page += 1

        # Render page to an image
        pix = page.get_pixmap()

        # Create temporary image file
        temp_image = TemporaryImageFile(pix)

        # Extract text from the page
        text = page.get_text()

        return temp_image, text


In [6]:
import copy
import tqdm
import json
import requests
import json
import base64

In [51]:
all_rez = dict()
pdf_iterator = PDFPageIterator("aami-pds.pdf")
for page_count, (image_path, text) in tqdm.tqdm(enumerate(pdf_iterator, start=1)):
    prompt = f'''
Create a structured HTML document based on the provided 
{text} , using the reference image to guide the HTML hierarchy and relationships. Your task is to analyze the image to understand the desired HTML layout, focusing on the hierarchy and grouping of elements. Then, process the given 
{text} to identify all information to be included in the HTML structure. Construct a valid, well-formatted HTML document that accurately represents the text content, using the image solely as a guide for structuring HTML elements. Add a `page_no` attribute to the `<body>` tag for each page represented in the image or text. Ensure that the HTML structure reflects the hierarchy and grouping shown in the image and preserves the original wording from the text input.

Please enclose the content within `<body>` tags and use separate `<body>` blocks with appropriate `page_no` attributes for multiple pages.

Example:

```html
<body page_no="1">
<!-- Content for page 1 -->
</body>

<body page_no="2">
<!-- Content for page 2 -->
</body>
```
    '''
    parameters = {
        'temperature': 0.0,  # Adjusts creativity; lower values yield more deterministic responses
        'num_predict': 1500
        
    }
    messages = [
        {
            'role': 'system',
            'content':"Your are an HTML Conversion BOT" ,
        },
        {
            'role': 'user',
            'content': prompt,
            'images': [image_path.filename]
        }
    ]

    # Send the request to the 'llava' model
    response = ollama.chat(model='llama3.2-vision', messages=messages, options=parameters)
    all_rez[page_count] = response['message']['content']

55it [58:59, 64.36s/it]


In [52]:
with open("aami-llama-13b-v.json", "w") as f:
        json.dump(all_rez, f)

In [56]:
print(all_rez[3])

To create a structured HTML document based on the provided text, we need to analyze the image and understand the desired HTML layout. The image shows a table with two columns: "What we cover" and "Page". We will use this information to create an HTML structure that accurately represents the text content.

Here is the HTML code:

```html
<!DOCTYPE html>
<html>
<head>
<title>Insurance Policy Document</title>
</head>
<body page_no="1">
<h1>What we Cover</h1>
<table>
  <tr>
    <th>Insured Events</th>
    <td>Flood (see important note on page 39)</td>
  </tr>
  <tr>
    <th>Legal Liability</th>
    <td>We cover your legal liability to pay compensation for death or bodily injury to other people, or loss or damage to their property.</td>
  </tr>
  <tr>
    <th>Additional Cover that Comes with Your Policy</th>
    <td>There are some additional covers that come with your policy for no extra cost. See section 5 'Additional cover that comes with your policy' on page 53 for more information.</td>

In [57]:
%%html
<!DOCTYPE html>
<html>
<head>
<title>Insurance Policy Document</title>
</head>
<body page_no="1">
<h1>What we Cover</h1>
<table>
  <tr>
    <th>Insured Events</th>
    <td>Flood (see important note on page 39)</td>
  </tr>
  <tr>
    <th>Legal Liability</th>
    <td>We cover your legal liability to pay compensation for death or bodily injury to other people, or loss or damage to their property.</td>
  </tr>
  <tr>
    <th>Additional Cover that Comes with Your Policy</th>
    <td>There are some additional covers that come with your policy for no extra cost. See section 5 'Additional cover that comes with your policy' on page 53 for more information.</td>
  </tr>
  <tr>
    <th>Optional Cover You Can Pay Extra For</th>
    <td>There are some optional covers that you have to pay extra for. If an optional cover applies to your policy it is shown on your certificate of insurance. See section 6 'Optional cover you can pay extra for' on page 65 for more information.</td>
  </tr>
  <tr>
    <th>Exclusion for New Policies</th>
    <td>We do not insure you for bushfire, storm, storm surge, flood or tsunami in the first 72 hours of your policy. Very limited exceptions apply. For full details see section 3 'Things we don't cover' on page 26.</td>
  </tr>
</table>
</body>

<body page_no="2">
<h1>Summary of Cover</h1>
<p>Limits, conditions and exclusions apply. Read your policy for full details.</p>
<table>
  <tr>
    <th>Insured Events</th>
    <td>Flood (see important note on page 39)</td>
  </tr>
  <tr>
    <th>Legal Liability</th>
    <td>We cover your legal liability to pay compensation for death or bodily injury to other people, or loss or damage to their property.</td>
  </tr>
</table>
</body>

<body page_no="3">
<h1>Additional Cover that Comes with Your Policy</h1>
<p>There are some additional covers that come with your policy for no extra cost. See section 5 'Additional cover that comes with your policy' on page 53 for more information.</p>
<table>
  <tr>
    <th>Optional Cover You Can Pay Extra For</th>
    <td>There are some optional covers that you have to pay extra for. If an optional cover applies to your policy it is shown on your certificate of insurance. See section 6 'Optional cover you can pay extra for' on page 65 for more information.</td>
  </tr>
</table>
</body>

<body page_no="4">
<h1>Exclusion for New Policies</h1>
<p>We do not insure you for bushfire, storm, storm surge, flood or tsunami in the first 72 hours of your policy. Very limited exceptions apply. For full details see section 3 'Things we don't cover' on page 26.</p>
<table>
  <tr>
    <th>Insured Events</th>
    <td>Flood (see important note on page 39)</td>
  </tr>
  <tr>
    <th>Legal Liability</th>
    <td>We cover your legal liability to pay compensation for death or bodily injury to other people, or loss or damage to their property.</td>
  </tr>
</table>
</body>

<body page_no="5">
<h1>Additional Cover that Comes with Your Policy</h1>
<p>There are some additional covers that come with your policy for no extra cost. See section 5 'Additional cover that comes with your policy' on page 53 for more information.</p>
<table>
  <tr>
    <th>Optional Cover You Can Pay Extra For</th>
    <td>There are some optional covers that you have to pay extra for. If an optional cover applies to your policy it is shown on your certificate of insurance. See section 6 'Optional cover you can pay extra for' on page 65 for more information.</td>
  </tr>
</table>
</body>

<body page_no="6">
<h1>Exclusion for New Policies</h1>
<p>We do not insure you for bushfire, storm, storm surge, flood or tsunami in the first 72 hours of your policy. Very limited exceptions apply. For full details see section 3 'Things we don't cover' on page 26.</p>
<table>
  <tr>
    <th>Insured Events</th>
    <td>Flood (see important note on page 39)</td>
  </tr>
  <tr>
    <th>Legal Liability</th>
    <td>We cover your legal liability to pay compensation for death or bodily injury to other people, or loss or damage to their property.</td>
  </tr>
</table>
</body>

<body page_no="7">
<h1>Additional Cover that Comes with Your Policy</h1>
<p>There are some additional covers that come with your policy for no extra cost. See section 5 'Additional cover that comes with your policy' on page 53 for more information.</p>
<table>
  <tr>
    <th>Optional Cover You Can Pay Extra For</th>
    <td>There are some optional covers that you have to pay extra for. If an optional cover applies to your policy it is shown on your certificate of insurance. See section 6 'Optional cover you can pay extra for' on page 65 for more information.</td>
  </tr>
</table>
</body>

<body page_no="8">
<h1>Exclusion for New Policies</h1>
<p>We do not insure you for bushfire, storm, storm surge, flood or tsunami in the first 72 hours of your policy. Very limited exceptions apply. For full details see section 3 'Things we don't cover' on page 26.</p>
<table>
  <tr>
    <th>Insured Events</th>
    <td>Flood (see important note on page 39)</td>
  </tr>
  <tr>
    <th>Legal Liability</th>
    <td>We cover your legal liability to pay compensation for death or bodily injury to other people, or loss or damage to their property.</td>
  </tr>
</table>
</body>

<body page_no="9">
<h1>Additional Cover that Comes with Your Policy</h1>
<p>There are some additional covers that come with your policy for no extra cost. See section 5

Insured Events,Flood (see important note on page 39)
Legal Liability,"We cover your legal liability to pay compensation for death or bodily injury to other people, or loss or damage to their property."
Additional Cover that Comes with Your Policy,There are some additional covers that come with your policy for no extra cost. See section 5 'Additional cover that comes with your policy' on page 53 for more information.
Optional Cover You Can Pay Extra For,There are some optional covers that you have to pay extra for. If an optional cover applies to your policy it is shown on your certificate of insurance. See section 6 'Optional cover you can pay extra for' on page 65 for more information.
Exclusion for New Policies,"We do not insure you for bushfire, storm, storm surge, flood or tsunami in the first 72 hours of your policy. Very limited exceptions apply. For full details see section 3 'Things we don't cover' on page 26."
Insured Events,Flood (see important note on page 39)
Legal Liability,"We cover your legal liability to pay compensation for death or bodily injury to other people, or loss or damage to their property."
Optional Cover You Can Pay Extra For,There are some optional covers that you have to pay extra for. If an optional cover applies to your policy it is shown on your certificate of insurance. See section 6 'Optional cover you can pay extra for' on page 65 for more information.
Insured Events,Flood (see important note on page 39)
Legal Liability,"We cover your legal liability to pay compensation for death or bodily injury to other people, or loss or damage to their property."
Optional Cover You Can Pay Extra For,There are some optional covers that you have to pay extra for. If an optional cover applies to your policy it is shown on your certificate of insurance. See section 6 'Optional cover you can pay extra for' on page 65 for more information.


In [37]:
print("""html\n<!DOCTYPE html>\n<html>\n<head>\n\t<title>HOME BUILDING INSURANCE</title>\n</head>\n<body page_no="1">\n\t<h1>HOME BUILDING INSURANCE</h1>\n\t<p>PRODUCT DISCLOSURE STATEMENT</p>\n\t<img src="living_room.jpg" alt="Living Room Photo">\n</body>\n\n<body page_no="2">\n\t<!-- Additional content for page 2 -->\n</body>\n""")

html
<!DOCTYPE html>
<html>
<head>
	<title>HOME BUILDING INSURANCE</title>
</head>
<body page_no="1">
	<h1>HOME BUILDING INSURANCE</h1>
	<p>PRODUCT DISCLOSURE STATEMENT</p>
	<img src="living_room.jpg" alt="Living Room Photo">
</body>

<body page_no="2">
	<!-- Additional content for page 2 -->
</body>



In [44]:
image_path.filename

'/var/folders/f3/dm3_bp0d7q7g3qlv3z2_ykgh0000gn/T/tmp8eg2dwjv.jpg'